In [ ]:
from connections.local_connection import sales_tracings_local as sl
from connections.remote_connection import rebate_tracings_collection as rb
import os
import pandas as pd
from rich import print

In [ ]:
months = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']
years = ['2021', '2022']

m_y = [f"{m.capitalize()} {y}" for m in months for y in years]

xl_to_pd_dict = {}

In [ ]:
mckesson_fp = os.path.join(r"//busse", "sales tracings", "mgm.xlsm")
assert os.path.exists(mckesson_fp), "Mckesson file not found"

mckesson = pd.DataFrame()

for sheet in m_y:
    try:
        df = pd.read_excel(mckesson_fp, sheet_name=sheet,)
        df['month'] = sheet
        mckesson = mckesson.append(df, ignore_index=True)
    except:
        print(f"Sheet {sheet} not found")
        

mckesson["distributor"] = "Mckesson"

In [ ]:
xl_to_pd_dict['mckesson'] = {
    'name': 'name',
    'addr1': 'addr1',
    'addr2': 'addr2',
    'city': 'addr3',        
    'state': 'state',
    'zip': 'i',
    'part': 'item',
    'quantity': 'qty',
    'uom': 'uom',
    'gross_price': 'cost',
    'net_price': 'q',
    'rebate': 'p',
    'rebate_contract': 'rebate',
    'date': 'date',
    'distributor': 'distributor',
}

In [ ]:
mckesson_df = mckesson[xl_to_pd_dict['mckesson'].values()].copy()
mckesson_df['date'] = pd.to_datetime(mckesson_df['date'], format='%Y%m%d')
mckesson_df.columns = xl_to_pd_dict['mckesson'].keys()

mckesson_df["addr2"].fillna("", inplace=True)
mckesson_df["rebate_contract"].fillna("", inplace=True)

mckesson_df.dtypes

# 144k rows

In [ ]:
xl_to_pd_dict['medline'] = {
    'name': 'CustName',
    'addr1': 'CustStreet',
    'addr2': 'tempa',
    'city': 'CustCity',        
    'state': 'CustState',
    'zip': 'CustZipcod',
    'part': 'VendorItm',
    'quantity': 'Quantity',
    'uom': 'UoM',
    'gross_price': 'AcqCost',
    'net_price': 'cost',
    'rebate': 'RebateAmt',
    'rebate_contract': 'VendorCont',
    'date': 'InvoiceDat',
    'distributor': 'distributor',
}

In [ ]:
medline_fp = os.path.join(r"//busse", "sales tracings", "medline.xlsm")
assert os.path.exists(medline_fp), "Medline file not found"

medline = pd.DataFrame()

for sheet in m_y:
    try:
        df = pd.read_excel(medline_fp, sheet_name=sheet,)
        df['month'] = sheet
        medline = medline.append(df, ignore_index=True)
    except:
        print(f"Sheet {sheet} not found")

medline["distributor"] = "Medline"

In [ ]:
medline_df = medline[xl_to_pd_dict['medline'].values()].copy()
medline_df.columns = xl_to_pd_dict['medline'].keys()
medline_df['date'] = pd.to_datetime(medline_df['date'], format='%Y%m%d')

medline_df["addr2"].fillna("", inplace=True)
medline_df["rebate_contract"].fillna("", inplace=True)

# medline specific
medline_df["gross_price"] = medline_df["gross_price"] * medline_df["quantity"]

medline_df.dtypes

# 162358 rows

In [ ]:
om_fp = os.path.join(r"//busse", "sales tracings", "om.xlsm")
assert os.path.exists(om_fp), "Om file not found"

om = pd.DataFrame()

for sheet in m_y:
    try:
        df = pd.read_excel(om_fp, sheet_name=sheet,)
        df['month'] = sheet
        om = om.append(df, ignore_index=True)
    except:
        print(f"Sheet {sheet} not found")

om["distributor"] = "Owens & Minor"

In [ ]:
xl_to_pd_dict['om'] = {
    'name': 'CUST NAME',
    'addr1': 'CUST ADDR 1',
    'addr2': 'CUST ADDR 2',
    'city': 'CITY       ',        
    'state': 'STATE',
    'zip': 'ZIP CODE            ',
    'part': 'PART NBR',
    'quantity': 'SHIP QTY',
    'uom': 'BU',
    'gross_price': 'NET UNIT COST          ',
    'net_price': 'INVOICE COST     ',
    'rebate': 'REBATE     ',
    # 'rebate': 'REBATE    ',
    'rebate_contract': 'COMMITMENT #',
    'date': 'INVOICE DT',
    'distributor': 'distributor',
}

In [ ]:
om_df = om[xl_to_pd_dict['om'].values()].copy()
om_df.columns = xl_to_pd_dict['om'].keys()

om_df["addr2"].fillna("", inplace=True)
om_df["rebate_contract"].fillna("", inplace=True)

def correct_date(x):
    if len(x) == 7:
        return f"0{x}"
    else:
        return x

# om specific
om_df["zip"] = om_df["zip"].astype(str)
om_df['date'] = om_df['date'].astype(str)
om_df["date"] = om_df["date"].apply(correct_date)
om_df['date'] = pd.to_datetime(om_df['date'], format='%m%d%Y')
om_df['quantity'] = om_df['quantity'].astype(float)
om_df['rebate'] = om_df["gross_price"] - om_df["net_price"]
om_df["rebate"] = om_df["rebate"] * om_df["quantity"]
om_df["gross_price"] = om_df["gross_price"] * om_df["quantity"]
om_df["net_price"] = om_df["net_price"] * om_df["quantity"]


om_df.dtypes

# 64274 rows

In [ ]:
xl_to_pd_dict['cardinal'] = {
    'name': 'Hosp',
    'addr1': 'Address',
    'addr2': 'Address',
    'city': 'City',        
    'state': 'St',
    'zip': 'Zip',
    'part': 'Item',
    'quantity': 'Qty',
    'uom': 'UOM',
    'gross_price': 'Ext Reb Amt',
    'net_price': 'Ext Reb Amt',
    'rebate': 'Ext Reb Amt',    
    'rebate_contract': 'Contract',
    'date': 'Bill Date',
    'distributor': 'distributor',
}

In [ ]:
cardinal_fp = os.path.join(r"//busse", "sales tracings", "cardinal.xlsm")
assert os.path.exists(om_fp), "Cardinal file not found"

cardinal = pd.DataFrame()

for sheet in m_y:
    try:
        df = pd.read_excel(cardinal_fp, sheet_name=f"{sheet} GPOData",)
        df['month'] = sheet
        cardinal = cardinal.append(df, ignore_index=True)
    except:
        print(f"Sheet {sheet} not found")

cardinal["distributor"] = "Cardinal"

In [ ]:
cardinal_df = cardinal[xl_to_pd_dict['cardinal'].values()].copy()
cardinal_df.columns = xl_to_pd_dict['cardinal'].keys()

cardinal_df["rebate_contract"].fillna("", inplace=True)

# cardinal specific
cardinal_df["addr2"] = ""
cardinal_df["part"] = cardinal_df["part"].astype(str)
cardinal_df.dropna(subset=["part"], inplace=True)
cardinal_df["part"] = cardinal_df["part"].str.replace(".0", "")
cardinal_df["date"] = cardinal_df["date"].astype(str)
cardinal_df["date"] = pd.to_datetime(cardinal_df["date"])
cardinal_df["gross_price"] = 0.00
cardinal_df["net_price"] = 0.00

cardinal_df.dtypes

# 139869 rows

In [ ]:
# get from rebate tracings March 2022 +
mdb_to_pd_dict = {
    'name': 'name',
    'addr1': 'addr',
    'addr2': 'addr', # set to ""
    'city': 'city',
    'state': 'state',
    'zip': 'state', # set to "" or use python lib to lookup zipcodes
    'part': 'part',
    'quantity': 'ship_qty',
    'uom': 'uom',
    'gross_price': 'cost', # set to 0.00
    'net_price': 'cost',
    'rebate': 'rebate',
    'rebate_contract': 'contract',
    'date': 'invoice_date',
    'distributor': 'distributor',
}

In [ ]:
import re

rebate_tracings = list(rb.find({
    "period": {
        "$not": {
            "$in": ["RETRO_MAY"]
        }
    }
}))

rebate_tracings = pd.DataFrame(rebate_tracings)

do_not_include = re.compile(r"medline|card|cardinal|om|mgm|mckesson|retro_may", re.IGNORECASE)

uniques = list(rebate_tracings["period"].unique())

uniques = [x for x in uniques if not do_not_include.search(x)]
uniques = [x.split("-")[1] for x in uniques]
uniques = list(set(uniques))

include = "|".join(uniques)

print(include)

rebate_tracings = list(rb.find({
    "period": {
        "$regex": include,
        "$options": "i"
    }
}))

rb_df = pd.DataFrame(rebate_tracings)


In [ ]:
rb_df["distributor"] = rb_df["period"].str.split("-").str[1]

def correct_distributor(x):
    if x.lower().startswith("conc"):
        return "Concordance"
    elif x.lower().startswith("mms"):
        return "Concordance"
    elif x.lower().startswith("american"):
        return "American Medical Supply"
    elif x.lower().startswith("atlantic"):
        return "Atlantic Medical Supply"
    elif x.lower().startswith("sentry"):
        return "Sentry Medical"
    elif x.lower().startswith("henry"):
        return "Henry Schein"
    elif x.lower().startswith("retro"):
            return "Henry Schein"
    elif x.lower().endswith("schein"):
        return "Medline"
    elif x.lower().startswith("ndc"):
        return "NDC Health"
    elif x.lower().startswith("tri"):
        return "Tri Anim"
    elif x.lower().startswith("deal"):
        return "Dealmed"
    elif x.lower().startswith("mohawk"):
        return "Mohawk Medical"        
    elif x.lower().startswith("howard"):
        return "Howard Medical"
    elif x.lower().startswith("avid"):
        return "Avid Medical"
    elif x.lower().startswith("twin"):
        return "Twin Medical"
    
    return x

# rb_df specific
rb_df["distributor"] = rb_df["distributor"].apply(correct_distributor)

rb_df_df = rb_df[mdb_to_pd_dict.values()].copy()
rb_df_df.columns = mdb_to_pd_dict.keys()

def correct_uom(x):
    if x == "CA":
        return "CS"
    
    return x

# rb_df_df specific
rb_df_df["addr2"] = ""
rb_df_df["part"] = rb_df_df["part"].astype(str)
rb_df_df["uom"] = rb_df_df["uom"].apply(correct_uom)
rb_df_df["gross_price"] = 0.00

rb_df_df["date"] = rb_df_df["date"].astype(str)
rb_df_df["date"] = pd.to_datetime(rb_df_df["date"], errors='coerce')


rb_df_df.dtypes

# 38850 rows

In [ ]:
# combine all of the dataframes
df = pd.concat([rb_df_df, medline_df, mckesson_df, cardinal_df, om_df], ignore_index=True)

df['addr1'] = df['addr1'].str.replace(r'[^a-zA-Z0-9 ]', '')
df['addr2'] = df['addr2'].str.replace(r'[^a-zA-Z0-9 ]', '')

df = df[(df["gross_price"] != 0.00) & (df["net_price"] != 0.00) & (df["rebate"] != 0.00)].copy()

In [ ]:
# save as parquet
p_df = df.copy()
p_df['name'] = p_df['name'].str.upper().astype("|S")
p_df['addr1'] = p_df['addr1'].str.upper().astype("|S")
p_df['addr2'] = p_df['addr2'].str.upper().astype("|S")
p_df['city'] = p_df['city'].str.upper().astype("|S")
p_df['state'] = p_df['state'].str.upper().astype("|S")
p_df['zip'] = p_df['zip'].str.upper().astype("|S")
p_df['part'] = p_df['part'].str.upper().astype("|S")
p_df['uom'] = p_df['uom'].str.upper().astype("|S")
p_df['distributor'] = p_df['distributor'].str.upper().astype("|S")
p_df['rebate_contract'] = p_df['rebate_contract'].str.upper().astype("|S")

parquet_fp = os.path.join(r"//busse", "sales tracings", "rebate_tracings_2021_11-2022_partial.parquet")
p_df.to_parquet(parquet_fp)

print(os.path.exists(parquet_fp), parquet_fp)


In [ ]:
# save to mongodb
mdb_df = df.copy()
mdb_df['date'] = mdb_df['date'].astype(str)
mdb_df['date'] = pd.to_datetime(mdb_df['date'], errors='coerce', format="%Y-%m-%d", utc=True)
mdb_df['part'] = mdb_df['part'].astype(str)
mdb_df['zip'] = mdb_df['zip'].astype(str)


def correct_gross_price(x):
    if x["gross_price"] == 0 and x["net_price"] != 0:
        return x["net_price"] + x["rebate"]

    return x["gross_price"]

from functools import lru_cache
from uszipcode import SearchEngine

search = SearchEngine()

def correct_zip(x):
    if len(x["zip"]) < 5:
        zipcode = hashable_correct_zip(city=x["city"], state=x["state"])
        if zipcode != "":
            return zipcode
        else:
            return "0" * (5 - len(x["zip"])) + x["zip"]
        
    return x["zip"]

@lru_cache(maxsize=1000)
def hashable_correct_zip(city, state):
    try:
        zipcode = search.by_city_and_state(city, state, returns=1)
    except:
        zipcode = ""
        
    if len(zipcode) > 0:
        return zipcode[0].zipcode

    return ""

mdb_df["zip"] = mdb_df.apply(correct_zip, axis=1)
mdb_df["gross_price"] = mdb_df.apply(correct_gross_price, axis=1)

sl.delete_many({})
sl.insert_many(mdb_df.to_dict('records'))

# mdb_df.dtypes


# 240447 rows